In [1]:
import sys
import numpy as np
import json
import os

sys.path.append('../')

from functions import *

In [11]:
prefixed = [filename for filename in os.listdir("./final dumps") if filename.startswith("bootstrap_main")]

results = []

for name in prefixed:
    with open(f"./final dumps/{name}", "r") as f:
        d = json.load(f)
        covariates = d["covariates"]
        median_lower = d["lower"]
        median_upper = d["upper"]
        results.append((covariates, median_lower, median_upper))

In [13]:
percentile = lambda p: np.percentile(np.array(covariates), p)

cef1 = lambda c, cov: c[0]*(1-cov[0])+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
n_cov1 = 4

def cef2(c, cov):
    low = int(cov[1] >= percentile(0) and cov[1] < percentile(91.5))
    high = int(cov[1] >= percentile(91.5) and cov[1] <= percentile(100))
    return c[0]*cov[0] + c[1]*(1-cov[0]) + c[2]*cov[0]*high + c[3]*(1-cov[0])*high
n_cov2 = 4

def cef3(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0]
n_cov3 = 6

def cef4(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0] + c[6]*feedback**3*(1-cov[0]) + \
        c[7]*feedback**3*cov[0]
n_cov3 = 8

In [19]:
def loss_function2(c):
    cef = lambda cov: cef2(c, cov)
    print(c)
    return get_loss_function(results[1][0], results[1][1], results[1][2], cef)

In [27]:
from time import perf_counter

s = perf_counter()
for i in range(700):
    i**3
print(perf_counter()-s)

0.0005822129987791413


In [22]:
from time import perf_counter

loss_function2([1,1,1,1])

[1, 1, 1, 1]


1085.0864849835634

In [16]:
estimates1 = []
estimates2 = []

for i, result in enumerate(results):
    
    def loss_function1(c):
        cef = lambda cov: cef1(c, cov)
        return get_loss_function(result[0], result[1], result[2], cef)
    
    point1, interval1 = get_estimates(loss_function1, n_cov1)
#     print(interval1)
    
    estimates1.append(interval1)
    
    
#     def loss_function2(c):
#         cef = lambda cov: cef2(c, cov)
#         return get_loss_function(result[0], result[1], result[2], cef)
    
#     point2, interval2 = get_estimates(loss_function2, n_cov2)
#     print(interval2)
    
#     estimates2.append(interval2)
    

In [17]:
a_lower = [e[0] for e in estimates1]
a_upper = [e[1] for e in estimates1]
a2_lower = [e[2] for e in estimates1]
a2_upper = [e[3] for e in estimates1]
b_lower = [e[4] for e in estimates1]
b_upper = [e[5] for e in estimates1]
b2_lower = [e[6] for e in estimates1]
b2_upper = [e[7] for e in estimates1]

print("95% confidence interval for a:")
print(f"[{np.percentile(a_lower, 2.5)}, {np.percentile(a_upper, 97.5)}]")

print("95% confidence interval for a2:")
print(f"[{np.percentile(a2_lower, 2.5)}, {np.percentile(a2_upper, 97.5)}]")

print("95% confidence interval for b:")
print(f"[{np.percentile(b_lower, 2.5)}, {np.percentile(b_upper, 97.5)}]")

print("95% confidence interval for b2:")
print(f"[{np.percentile(b2_lower, 2.5)}, {np.percentile(b2_upper, 97.5)}]")

95% confidence interval for a:
[-1.1865814956398697, -0.2569725435499196]
95% confidence interval for a2:
[-3.354556926711658, 0.008372027502344021]
95% confidence interval for b:
[0.02163135485781623, 0.15772463467297296]
95% confidence interval for b2:
[-0.007395051600975394, 0.308763513574974]


In [31]:
c0_lower1 = [e[0] for e in estimates1]
c0_upper1 = [e[1] for e in estimates1]
c1_lower1 = [e[2] for e in estimates1]
c1_upper1 = [e[3] for e in estimates1]
c2_lower1 = [e[4] for e in estimates1]
c2_upper1 = [e[5] for e in estimates1]
c3_lower1 = [e[6] for e in estimates1]
c3_upper1 = [e[7] for e in estimates1]

In [34]:
print(f"c0: [{np.percentile(c0_lower1, 2.5)}, {np.percentile(c0_upper1, 97.5)}]")
print(f"c1: [{np.percentile(c1_lower1, 2.5)}, {np.percentile(c1_upper1, 97.5)}]")
print(f"c2: [{np.percentile(c2_lower1, 2.5)}, {np.percentile(c2_upper1, 97.5)}]")
print(f"c3: [{np.percentile(c3_lower1, 2.5)}, {np.percentile(c3_upper1, 97.5)}]")

c0: [-1.1438769546955532, -0.21484968075437189]
c1: [-3.007696430111329, -0.2498179077796555]
c2: [0.031010737447792085, 0.14820856783630998]
c3: [0.009367319324892874, 0.2720486568158093]


In [35]:
print(f"c0: [{np.percentile(c0_lower1, 5)}, {np.percentile(c0_upper1, 95)}]")
print(f"c1: [{np.percentile(c1_lower1, 5)}, {np.percentile(c1_upper1, 95)}]")
print(f"c2: [{np.percentile(c2_lower1, 5)}, {np.percentile(c2_upper1, 95)}]")
print(f"c3: [{np.percentile(c3_lower1, 5)}, {np.percentile(c3_upper1, 95)}]")

c0: [-1.0824604248970362, -0.3167180977774851]
c1: [-2.887517630330213, -0.28083685174924744]
c2: [0.040184967400121174, 0.1359385262146973]
c3: [0.011351974696634532, 0.2601844999718078]


In [ ]:
c0_lower2 = [e[0] for e in estimates2]
c0_upper2 = [e[1] for e in estimates2]
c1_lower2 = [e[2] for e in estimates2]
c1_upper2 = [e[3] for e in estimates2]
c2_lower2 = [e[4] for e in estimates2]
c2_upper2 = [e[5] for e in estimates2]
c3_lower2 = [e[6] for e in estimates2]
c3_upper2 = [e[7] for e in estimates2]

In [ ]:
print(f"c0: [{np.percentile(c0_lower2, 2.5)}, {np.percentile(c0_upper2, 97.5)}]")
print(f"c1: [{np.percentile(c1_lower2, 2.5)}, {np.percentile(c1_upper2, 97.5)}]")
print(f"c2: [{np.percentile(c2_lower2, 2.5)}, {np.percentile(c2_upper2, 97.5)}]")
print(f"c3: [{np.percentile(c3_lower2, 2.5)}, {np.percentile(c3_upper2, 97.5)}]")

In [9]:
def loss_function2(c):
    cef = lambda cov: cef2(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point2, interval2 = get_estimates(loss_function2, n_cov2)

print(point2)
print(interval2)

[-0.62927241 -0.68833565  0.2191079   0.21371192]
[-0.6310305399130508, -0.6275197196464853, -0.6903548399814075, -0.6863105212481744, 0.21664061440503107, 0.22157520490644744, 0.21097234125906592, 0.2164515191710961]


In [ ]:
def loss_function3(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point3, interval3 = get_estimates(loss_function3, n_cov3)

print(point3)
print(interval3)

In [ ]:
def loss_function4(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point4, interval4 = get_estimates(loss_function4, n_cov4)

print(point4)
print(interval4)